In [2]:
import pygame
import numpy as np
from Manobras import *
from Orbitas import *


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:

if __name__ == "__main__":

    DA = angulo_manobra(Orbita_Nave, Orbita_Detrito, 0)

    Ra = posicao_orbita(Orbita_Detrito, angulo_manobra(Orbita_Detrito, Orbita_Nave, 0))
    Rp = posicao_orbita(Orbita_Nave, angulo_manobra(Orbita_Nave, Orbita_Detrito, 0))

    e = (Ra - Rp) / (Ra + Rp)
    Periapse_angle = (DA + Orbita_Nave[4]) % (2*np.pi)
    a = Rp / (1 - e)

    Orbita_Transferencia = [
        a,
        e,
        Orbita_Nave[2],   # RAAN
        Orbita_Nave[3],   # Inclinação
        Periapse_angle
    ]

    print("=== ORBITA INTERMEDIARIA ===")
    print(f"e = {e}")
    print(f"a = {a}")
    print(f"ω = {Periapse_angle}")
    print("Orbita =", Orbita_Transferencia)


=== ORBITA INTERMEDIARIA ===
e = 0.02403732603208904
a = 6989.24125572833
ω = 0.22809053044783223
Orbita = [6989.24125572833, 0.02403732603208904, 5.051777, 0.7504916, np.float64(0.22809053044783223)]


In [8]:
# ================= PYGAME ==================
pygame.init()
WIDTH, HEIGHT = 900, 900
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Orbital Simulation")

CENTER = np.array([WIDTH//2, HEIGHT//2])
SCALE = 0.05  # Ajuste para caber na tela

# ================= FUNÇÃO PARA DESENHAR ORBITA ==================
def draw_orbit(orbit, color):
    a, e, RAAN, inc, w = orbit[0], orbit[1], orbit[2], orbit[3], orbit[4]
    
    points = []
    for theta in np.linspace(0, 2*np.pi, 500):
        r = a*(1-e**2)/(1 + e*np.cos(theta))
        
        # posição no plano orbital
        x = r * np.cos(theta + w)
        y = r * np.sin(theta + w)
        
        # escala e centro
        pos = CENTER + SCALE * np.array([x, -y])
        points.append(pos)

    pygame.draw.lines(screen, color, True, points, 2)

# ================= LOOP ==================
running = True
while running:
    screen.fill((0,0,0))

    # Terra
    pygame.draw.circle(screen, (0,100,255), CENTER, 10)

    # Orbitas
    draw_orbit(Orbita_Detrito, (255,0,0))
    draw_orbit(Orbita_Intermediaria, (0,255,0))
    draw_orbit(Orbita_Intermediaria2, (255,255,0))
    draw_orbit(Orbita_Nave, (0,255,255))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    pygame.display.flip()

pygame.quit()


In [7]:

WIDTH, HEIGHT = 1000, 1000
CENTER = np.array([WIDTH//2, HEIGHT//2])
SCALE = 0.05  # km → pixels

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simulador Orbital 3D")

def projecao_3D_para_2D(r):
    # projeção isométrica simples
    x = r[0]
    y = r[1]
    z = r[2]
    return np.array([
        x - y,
        (x + y)/2 - z
    ])

def desenhar_orbita(orbita, color):
    pontos = []
    for f in np.linspace(0, 2*np.pi, 500):
        r = posicao_xyz(orbita, f).flatten()
        p2d = projecao_3D_para_2D(r) * SCALE + CENTER
        pontos.append(p2d)
    pygame.draw.lines(screen, color, True, pontos, 2)

# LOOP
running = True
while running:
    screen.fill((0,0,0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    desenhar_orbita(Orbita_Nave, (0,255,0))
    desenhar_orbita(Orbita_Detrito, (255,0,0))

    pygame.display.flip()

pygame.quit()

In [9]:


Orbita_Nave   = [6820, 0.0002, 5.05, 0.75, 3.80]
Orbita_Detrito = [7245, 0.0121, 5.16, 1.23, 3.36]

WIDTH, HEIGHT = 900, 900
CENTER = np.array([WIDTH//2, HEIGHT//2])
SCALE = 0.05  # km → pixels

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))

def projecao_iso(r):
    x,y,z = r
    return np.array([x - y, (x+y)/2 - z])

def desenhar_orbita(orbita, color):
    pontos=[]
    for f in np.linspace(0,2*np.pi,600):
        r = posicao_xyz(orbita,f).flatten()
        p = projecao_iso(r)*SCALE + CENTER
        pontos.append(p)
    pygame.draw.lines(screen,color,True,pontos,2)

# LOOP
running=True
while running:
    screen.fill((0,0,0))

    for e in pygame.event.get():
        if e.type == pygame.QUIT:
            running=False

    desenhar_orbita(Orbita_Nave,(0,255,0))
    desenhar_orbita(Orbita_Detrito,(255,0,0))
    pygame.draw.circle(screen,(255,255,0),CENTER.astype(int),6)

    pygame.display.flip()

pygame.quit()


In [ ]:
def desenhar_orbita(screen, orbita, color, scale, center):
    pts = []
    for f in np.linspace(0, 2*np.pi, 500):
        R = posicao_xyz(orbita, f)
        x = (R[0] - R[1]) * scale + center[0]
        y = ((R[0] + R[1])/2 - R[2]) * scale + center[1]
        pts.append((x,y))
    pygame.draw.lines(screen, color, True, pts, 2)


In [ ]:

pygame.init()

W,H = 1200,1200
screen = pygame.display.set_mode((W,H))
pygame.display.set_caption("Simulador Orbital com Manobras")

CENTER = np.array([W//2, H//2])
SCALE = 0.03

clock = pygame.time.Clock()
f = 0

running = True
while running:
    dt = clock.tick(60)/1000
    f += 0.002

    for e in pygame.event.get():
        if e.type == pygame.QUIT:
            running = False

    screen.fill((0,0,0))

    # Terra
    pygame.draw.circle(screen,(100,100,255),CENTER,12)

    # Orbitas
    desenhar_orbita(screen, Orbita_Nave, (0,255,0), SCALE, CENTER)
    desenhar_orbita(screen, Orbita_Detrito, (255,0,0), SCALE, CENTER)

    # Nave
    Rn = posicao_xyz(Orbita_Nave, f)
    x = (Rn[0]-Rn[1])*SCALE + CENTER[0]
    y = ((Rn[0]+Rn[1])/2 - Rn[2])*SCALE + CENTER[1]
    pygame.draw.circle(screen,(0,255,255),(int(x),int(y)),5)

    # Detrito
    Rd = posicao_xyz(Orbita_Detrito, f*0.7)
    xd = (Rd[0]-Rd[1])*SCALE + CENTER[0]
    yd = ((Rd[0]+Rd[1])/2 - Rd[2])*SCALE + CENTER[1]
    pygame.draw.circle(screen,(255,255,0),(int(xd),int(yd)),5)

    # Manobra no ponto f=π/3
    desenhar_manobra(screen, Orbita_Nave, Orbita_Detrito, np.pi/3, SCALE, CENTER)

    pygame.display.flip()

pygame.quit()


IndexError: index 1 is out of bounds for axis 0 with size 1

: 